In [2]:
! pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.3/901.3 kB 50.5 MB/s eta 0:00:00


In [3]:
from ultralytics import YOLO
import os
import yaml
import shutil
from pathlib import Path

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
def prepare_dataset_yaml():
    # Get class names from classes.txt
    with open('YOLO_labels/classes.txt', 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    # Create train and val directories
    dataset_path = Path('YOLO_dataset')
    for split in ['train', 'val']:
        (dataset_path / split / 'images').mkdir(parents=True, exist_ok=True)
        (dataset_path / split / 'labels').mkdir(parents=True, exist_ok=True)

    # Create data.yaml
    data_yaml = {
        'path': str(dataset_path.absolute()),
        'train': 'train/images',
        'val': 'val/images',
        'nc': len(classes),
        'names': classes
    }

    with open(dataset_path / 'data.yaml', 'w') as f:
        yaml.dump(data_yaml, f, sort_keys=False)

    return data_yaml, classes

In [5]:
def organize_dataset():
    """Organize dataset into train/val splits"""
    dataset_path = Path('YOLO_dataset')

    # Get all image files from Dataset directory
    image_files = []
    for class_dir in Path('Hand Gestures Dataset').iterdir():
        if class_dir.is_dir():
            image_files.extend(list(class_dir.glob('*.jpg')))
            image_files.extend(list(class_dir.glob('*.jpeg')))
            image_files.extend(list(class_dir.glob('*.png')))

    # Split into train/val (0.90/0.10)
    import random
    random.shuffle(image_files)
    split_idx = int(len(image_files) * 0.90)
    train_files = image_files[:split_idx]
    val_files = image_files[split_idx:]

    # Copy files to train and val directories
    for img_path in train_files:
        # Copy image
        shutil.copy2(img_path, dataset_path / 'train' / 'images' / img_path.name)
        # Copy corresponding label from YOLO_labels directory
        label_path = Path('YOLO_labels') / img_path.parent.name / f"{img_path.stem}.txt"
        if label_path.exists():
            shutil.copy2(label_path, dataset_path / 'train' / 'labels' / f"{img_path.stem}.txt")

    for img_path in val_files:
        # Copy image
        shutil.copy2(img_path, dataset_path / 'val' / 'images' / img_path.name)
        # Copy corresponding label from YOLO_labels directory
        label_path = Path('YOLO_labels') / img_path.parent.name / f"{img_path.stem}.txt"
        if label_path.exists():
            shutil.copy2(label_path, dataset_path / 'val' / 'labels' / f"{img_path.stem}.txt")

    print(f"Total images: {len(image_files)}")
    print(f"Training images: {len(train_files)}")
    print(f"Validation images: {len(val_files)}")

In [11]:
!gdown 1z9NPa2ljz4LoopboecUp14T4IxzZVfZf
!unzip HandGesturesDataset.zip

Downloading...
From (original): https://drive.google.com/uc?id=1z9NPa2ljz4LoopboecUp14T4IxzZVfZf
From (redirected): https://drive.google.com/uc?id=1z9NPa2ljz4LoopboecUp14T4IxzZVfZf&confirm=t&uuid=058fb494-dd67-4790-b227-eaee9ea87592
To: /content/HandGesturesDataset.zip
100% 291M/291M [00:02<00:00, 142MB/s]


In [13]:
!gdown 1KIhB1EJwQzvkl8V3HQb33-y2xNSHNNdc
!unzip labelImgOutput.zip

Downloading...
From: https://drive.google.com/uc?id=1KIhB1EJwQzvkl8V3HQb33-y2xNSHNNdc
To: /content/labelImgOutput.zip
100% 1.64M/1.64M [00:00<00:00, 172MB/s]
Archive:  labelImgOutput.zip
   creating: labelImg Output/
   creating: labelImg Output/Collab_VFR/
  inflating: labelImg Output/Collab_VFR/Collab_VFR_001.xml  
  inflating: labelImg Output/Collab_VFR/Collab_VFR_002.xml  
  inflating: labelImg Output/Collab_VFR/Collab_VFR_003.xml  
  inflating: labelImg Output/Collab_VFR/Collab_VFR_004.xml  
  inflating: labelImg Output/Collab_VFR/Collab_VFR_005.xml  
  inflating: labelImg Output/Collab_VFR/Collab_VFR_006.xml  
  inflating: labelImg Output/Collab_VFR/Collab_VFR_007.xml  
  inflating: labelImg Output/Collab_VFR/Collab_VFR_008.xml  
  inflating: labelImg Output/Collab_VFR/Collab_VFR_009.xml  
  inflating: labelImg Output/Collab_VFR/Collab_VFR_010.xml  
  inflating: labelImg Output/Collab_VFR/Collab_VFR_011.xml  
  inflating: labelImg Output/Collab_VFR/Collab_VFR_012.xml  
  inflatin

In [15]:
import xml.etree.ElementTree as ET
import glob
import os
from tqdm import tqdm

dataset_path = 'labelImg Output'
output_base = 'YOLO_labels'


def convert_xml_to_yolo(xml_path, class_names):
    """Convert single XML file to YOLO format"""
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Get image dimensions
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)

    result = []
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        # Get class index from class_names list
        class_id = class_names.index(class_name)

        # Get bounding box coordinates
        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)

        # Convert to YOLO format (normalized coordinates)
        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        w = (xmax - xmin) / width
        h = (ymax - ymin) / height

        # Format: class_id x_center y_center width height
        result.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

    return result


def create_yolo_dataset():
    class_names = [folder for folder in os.listdir(dataset_path)
                   if os.path.isdir(os.path.join(dataset_path, folder))]

    print(f"Found classes: {class_names}")

    os.makedirs(output_base, exist_ok=True)

    # Create classes.txt
    with open(os.path.join(output_base, 'classes.txt'), 'w') as f:
        f.write('\n'.join(class_names))

    # Process each class
    for class_name in class_names:
        print(f"\nProcessing class: {class_name}")

        # Create output directory for this class
        os.makedirs(os.path.join(output_base, class_name), exist_ok=True)

        # Get all XML files for this class
        xml_files = glob.glob(os.path.join(dataset_path, class_name, '*.xml'))

        # Process each XML file
        for xml_file in tqdm(xml_files, desc=f'Converting {class_name}'):
            try:
                # Convert XML to YOLO format
                yolo_annotations = convert_xml_to_yolo(xml_file, class_names)

                # Get base filename
                base_name = os.path.splitext(os.path.basename(xml_file))[0]

                # Save YOLO format annotation
                output_path = os.path.join(output_base, class_name, f'{base_name}.txt')
                with open(output_path, 'w') as f:
                    f.write('\n'.join(yolo_annotations))

            except Exception as e:
                print(f"Error processing {xml_file}: {str(e)}")






In [16]:
create_yolo_dataset()
print("\nConversion completed!")

total_files = sum([len(files) for r, d, files in os.walk(output_base)])
print(f"\nTotal converted files: {total_files}")
print(f"Output directory: {os.path.abspath(output_base)}")

Found classes: ['One_VFR', 'Nine_VFR', 'Span_VFR', 'Six_VFR', 'Eight_VFR', 'XSign_VFR', 'Four_VFR', 'Punch_VFR', 'TimeOut_VFR', 'Collab_VFR', 'Three_VFR', 'Horiz_VFR', 'Five_VFR', 'Seven_VFR', 'Two_VFR']

Processing class: One_VFR


Converting One_VFR: 100%|██████████| 200/200 [00:00<00:00, 6575.17it/s]



Processing class: Nine_VFR


Converting Nine_VFR: 100%|██████████| 200/200 [00:00<00:00, 7427.82it/s]



Processing class: Span_VFR


...


Processing class: Seven_VFR


Converting Seven_VFR: 100%|██████████| 200/200 [00:00<00:00, 7449.59it/s]



Processing class: Two_VFR


Converting Two_VFR: 100%|██████████| 200/200 [00:00<00:00, 7274.77it/s]


Conversion completed!

Total converted files: 3001
Output directory: /content/YOLO_labels


In [17]:
data_yaml, classes = prepare_dataset_yaml()
organize_dataset()

Total images: 3000
Training images: 2700
Validation images: 300


In [18]:
from google.colab import drive
drive.mount('/content/drive')
save_dir = '/content/drive/My Drive/YOLO/checkpoints'
os.makedirs(save_dir, exist_ok=True)

Mounted at /content/drive


In [19]:
def train_yolo():
    try:
        # Get dataset path
        dataset_path = Path('YOLO_dataset')

        # Training configuration
        config = {
            'data': str(dataset_path / 'data.yaml'),
            'epochs': 100,
            'batch': 16,
            'imgsz': 640,
            'patience': 20,
            'device': '0',  # Use '0' for GPU
            'workers': 8,
            'name': 'gesture_detection',
            'save_period': 10,  # Save checkpoint every 10 epochs
            'save': True  # Enable saving
        }

        model = YOLO('yolo11l.pt')
        # Define a custom callback to save the model after each epoch
        class SaveModelCallback:
            def __init__(self, save_dir):
                self.save_dir = save_dir

            def on_epoch_end(self, epoch, results):
                # Save model checkpoint to Google Drive
                model_path = os.path.join(self.save_dir, f'epoch_{epoch + 1}.pt')
                print(f"Saving model to {model_path}")
                model.save(model_path)

        # Attach the callback
        callback = SaveModelCallback(save_dir)
        model.add_callback('on_epoch_end', callback.on_epoch_end)

        results = model.train(**config)

        return model, results

    except Exception as e:
        print(f"Training error: {str(e)}")
        raise

In [20]:
print("Starting YOLO training...")
model, results = train_yolo()
print("\nTraining completed successfully!")



Starting YOLO training...


100%|██████████| 49.0M/49.0M [00:01<00:00, 44.9MB/s]


Ultralytics 8.3.51 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=YOLO_dataset/data.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=0, workers=8, project=None, name=gesture_detection, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

100%|██████████| 755k/755k [00:00<00:00, 133MB/s]


Overriding model.yaml nc=80 with nc=15

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  7                  -1  1   2360320  ultralytic

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      11.1G     0.2318     0.1218     0.8262         12        640: 100%|██████████| 169/169 [02:02<00:00,  1.38it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.48it/s]


                   all        300        300      0.999          1      0.995      0.977

100 epochs completed in 3.613 hours.
Optimizer stripped from runs/detect/gesture_detection/weights/last.pt, 51.2MB
Optimizer stripped from runs/detect/gesture_detection/weights/best.pt, 51.2MB

Validating runs/detect/gesture_detection/weights/best.pt...
Ultralytics 8.3.51 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11l summary (fused): 464 layers, 25,290,877 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:06<00:00,  1.54it/s]


                   all        300        300      0.999          1      0.995      0.977
               One_VFR         16         16          1          1      0.995      0.966
              Nine_VFR         25         25          1          1      0.995       0.99
              Span_VFR         14         14      0.994          1      0.995      0.995
               Six_VFR         23         23      0.997          1      0.995      0.986
             Eight_VFR         19         19          1          1      0.995      0.972
             XSign_VFR         17         17      0.997          1      0.995      0.961
              Four_VFR         25         25      0.997          1      0.995      0.991
             Punch_VFR         23         23          1          1      0.995      0.979
           TimeOut_VFR         11         11          1          1      0.995      0.941
            Collab_VFR         20         20      0.997          1      0.995      0.955
             Three_VF

In [25]:
# def save_model(model, output_dir):
#     try:
#         # Create output directory
#         os.makedirs(output_dir, exist_ok=True)
#
#         # 1. Save best model (from training)
#         best_model_path = 'best.pt'
#         # Copy the best weights file that was saved during training
#         if hasattr(model, 'best') and model.best:
#             shutil.copy(model.best, best_model_path)
#             print(f"\nBest model saved to: {best_model_path}")
#
#         # 2. Save last model (final state)
#         last_model_path = 'last.pt'
#         model.save(last_model_path)
#         print(f"Last model saved to: {last_model_path}")
#
#         # 3. Save model in ONNX format
#         try:
#             onnx_path = os.path.join(output_dir, 'model.onnx')
#             model.export(format='onnx', imgsz=640)
#             print(f"ONNX model saved to: {onnx_path}")
#         except Exception as e:
#             print(f"ONNX export failed: {str(e)}")
#
#         return True
#
#     except Exception as e:
#         print(f"Error saving model: {str(e)}")
#         return False

# output_dir = 'saved_models'
# save_model(model=model, output_dir=output_dir)

In [23]:
# Additional information
print("\nModel Information:")
print(f"Model Type: {model.type}")
print(f"Model Task: {model.task}")
print(f"Model Classes: {model.names}")


Model Information:
Model Type: <bound method Module.type of YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): B

In [27]:
# Print model summary
print("\nModel Summary:")
model.info()



Model Summary:
YOLO11l summary: 631 layers, 25,322,045 parameters, 0 gradients, 87.3 GFLOPs


(631, 25322045, 0, 87.33393919999999)